---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [32]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [45]:
import re
from datetime import date

def four_digit_year(year):
    if (year < 100):
        return year + 1900
    return year

month_resolver = {"Jan": 1, "January": 1, "Janaury": 1, "Feb": 2, "February": 2, "Mar": 3, "March": 3, "Apr": 4, "April": 4,
                  "May": 5, "Jun": 6, "June": 6, "Jul": 7, "July": 7, "Aug": 8, "August": 8, "Sep": 9, 
                  "September": 9, "Oct": 10,
                  "October": 10, "Nov": 11, "November": 11, "Dec": 12, "December": 12, "Decemeber": 12}
def str_to_month(text):
    return month_resolver[text]

def months_regex():
    return "(" + "|".join(month_resolver.keys()) + ").?"

def to_date(line):
    r = re.search("(\d{1,2}) ?" + months_regex() + " (\d{2,4})", line)
    if (r):
        return date(four_digit_year(int(r.group(3))),str_to_month(r.group(2)), int(r.group(1)))
    
    r = re.search(months_regex() + " (\d{1,2}),? (\d{2,4})", line)
    if (r):
        return date(four_digit_year(int(r.group(3))),str_to_month(r.group(1)), int(r.group(2)))
    
    r = re.search(months_regex() + " (\d{2,4})", line)
    if (r):
        return date(four_digit_year(int(r.group(2))),str_to_month(r.group(1)), 1)
    
    r = re.search("(\d{1,2})[/|-](\d{1,2})[/|-](\d{2,4})", line)
    if (r):
        return date(four_digit_year(int(r.group(3))),int(r.group(1)), int(r.group(2)))

    r = re.search("(\d{1,2})[/|-](\d{2,4})", line)
    if (r):
        month = int(r.group(1))
        if ((month >= 1) & (month <= 12)):
            return date(four_digit_year(int(r.group(2))),int(r.group(1)), 1)
    
    r = re.search("(\d{4})", line)
    if (r):
        return date(four_digit_year(int(r.group(1))),1, 1)
    
    raise NotImplementedError(line)

In [46]:
def date_sorter():
    
    dates = df.map(to_date)
    #for i in df:
    #    print(i, to_date(i))
    return pd.Series(dates.sort_values().index.values)

date_sorter()

03/25/93 Total time of visit (in minutes):
 1993-03-25
6/18/85 Primary Care Doctor:
 1985-06-18
sshe plans to move as of 7/8/71 In-Home Services: None
 1971-07-08
7 on 9/27/75 Audit C Score Current:
 1975-09-27
2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7
 1996-02-06
.Per 7/06/79 Movement D/O note:
 1979-07-06
4, 5/18/78 Patient's thoughts about current substance abuse:
 1978-05-18
10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview
 1989-10-24
3/7/86 SOS-10 Total Score:
 1986-03-07
(4/10/71)Score-1Audit C Score Current:
 1971-04-10
(5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC_12.6Activities of Daily Living (ADL) Bathing: Independent
 1985-05-11
4/09/75 SOS-10 Total Score:
 1975-04-09
8/01/98 Communication with referring physician?: Done
 1998-08-01
1/26/72 Communication with referring physician?: Not Done
 1972-01-26
5/24/1990 CPT Code: 90792: With medical services
 1990-05-24
1/25/2011 CPT Code: 90792: With medical services
 2011-01-25
4/12/82 Total time of

sS/P TAH/BSO : 7/2004 for endometrial hyperplasia
 2004-07-01
Retired in 11/1984 after teaching for 30 yearsLiving Situation Current Living Situation (Including type of dwelling and who patient lives with):
 1984-11-01
6e. monitor pt's depressive experience in context of adjusting to upcoming retirement from work in 5/2016 and realtionships with his brother, son and daughter This visit for a one-time consultation only? No
 2016-05-01
e12/2007 dx of endocarditis
 2007-12-01
11/1982 Primary Care Doctor:
 1982-11-01
06/1981 Hx of Brain Injury: Yes
 1981-06-01
CBC, CMP, TSH, EKG: re: QTc 445 ms in 4/2013 Collateral information obtained:
 2013-04-01
s Pt. is a 76-y.o. WWW who presents in the context of after being divorced in a contentious situation, married a man who was "a much beter match" for her and after his death in 4/1999 she has experienced a complicated grief reaction.
 1999-04-01
a Endometriosis : dx on laparoscopy 5/2006
 2006-05-01
10/1978 Communication with referring physician

0      271
1      272
2        9
3       84
4        2
5       53
6       28
7      474
8      153
9       13
10     129
11      98
12     111
13     225
14      31
15     171
16     191
17     486
18     335
19     415
20      36
21     323
22     405
23     422
24     375
25     380
26     345
27      57
28     481
29     436
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    286
477    244
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64